# to-do list
1) extract feature
2) label ([exc, hap, sur] 21%,[neu, fru] 42%,[fea, ang, sad, dis] 38%) -> 1 file.csv
3) process dims (double), bias (remove)
- add noise
- process real data

In [3]:
# !pip install torchaudio
# !pip install torch
# !pip install soundfile
#!pip install PySoundFile
# print(torchaudio.__version__)
# print(torch.__version__)
#!pip install numpy

In [27]:
import os
import torchaudio
from io import StringIO
import torch
import sndhdr
import soundfile
import numpy
import csv,glob
import pandas as pd

## 1) EXTRACT FEATURE

In [28]:
#Đọc link file
def read_link_wav(type = ('sentences', 'dialog'), sessions = [1,2,3,4,5]):
    link = os.path.join('IEMOCAP_full_release', 'Session'+str(sessions), type, 'wav')
    links = {}
    if type == 'dialog':
        return
    for dialog in os.listdir(link):
        list_sentence = []
        for sentence in os.listdir(os.path.join(link,dialog)):
            sentence_link = os.path.join(link, dialog, sentence)
            list_sentence.append(sentence_link)
        links[dialog] = list_sentence
    return links

In [29]:

#.wav to .pt
def read_and_save_mfcc(types = ('dialog','sentences')):
    for session in range(1,6):
        links = read_link_wav(types, session)
        root = os.path.join('IEMOCAP_full_release','Session'+str(session),types)
        if os.path.exists(os.path.join(root,'mfcc')):
            continue
        os.makedirs(os.path.join(root,'mfcc'))
        for dia in links.keys():
            if os.path.exists(os.path.join(root,'mfcc',dia)):
                continue
            os.makedirs(os.path.join(root,'mfcc',dia))
            for sen in links[dia]:
                try:
                    if sndhdr.what(sen).filetype == 'wav':
                        waveform, sample_rate = torchaudio.load(sen)
                        transform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate)
                        mel_specgram = transform(waveform)
                        output_file = os.path.join(root,'mfcc',dia,os.path.basename(sen)[:-4]+'.pt')
                        torch.save(mel_specgram,output_file)
                except:
                    pass

In [30]:
# # extract feature .wav to .pt
# read_and_save_mfcc('sentences')

## 2) Label + remove noise

In [33]:

def get_shape(file):
    mfcc = torch.load(file)
    return list(mfcc.shape)
def create_label_file():
    categories = {'positive': ['exc', 'hap', 'sur'],
                'neutral': ['neu','fru'],
                'negative': ['fea','ang','sad','dis']}
    header = ['sessionID','labels','positive','neutral','negative','dims','time_shape','path']
    with open('processed_label_data.csv',mode='w',newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
    for session in range(1,6):
        root = os.path.join('IEMOCAP_full_release','Session'+str(session),'dialog','EmoEvaluation','*')
        files = glob.glob(root)
        for file in files:
            if file[-3:] =='txt': 
                with open(file, 'r') as lines:
                    for line in lines:
                        words = line.split("\t")
                        update_sample(categories,session,words)
def update_sample(categories,session,words):
    if len(words)==4:
        path_file = os.path.join('IEMOCAP_full_release','Session'+str(session),'sentences','mfcc',words[1][:-5],words[1]+'.pt')
        if os.path.exists(path_file):
            shape = get_shape(path_file)
            with open('processed_label_data.csv',mode = 'a',newline = '') as f:
                write = csv.writer(f)
                if (words[2]) in categories['positive']:
                    write.writerow([words[1],words[2],1,0,0,shape,shape[2:],path_file])
                if (words[2]) in categories['neutral']:
                    write.writerow([words[1],words[2],0,1,0,shape,shape[2:],path_file])
                if (words[2]) in categories['negative']:
                    write.writerow([words[1],words[2],0,0,1,shape,shape[2:],path_file])
                if (words[2]) in ['xxx','oth']:
                    os.remove(path_file)

In [34]:
# create_label_file()

## 3) PREPROCESS DATA

In [ ]:
#check for mono file audio or not
def check_mono():
    for session in range(1,6):
        root = os.path.join('IEMOCAP_full_release','Session'+str(session),'sentences','mfcc')
        for dialog in os.listdir(root):
            for sentence in os.listdir(os.path.join(root,dialog)):
                sample = torch.load(os.path.join(root,dialog,sentence))
                if sample.shape[0]!=1:
                    return False
    return True

In [35]:
def reform_file_data():
    df = pd.read_csv('processed_label_data.csv')
    df['dims'] = df['dims'].apply(lambda x: [int(i) for i in x.strip('[]').split(',')])
    if str(df['time_shape'].dtype) != 'int64':
        df['time_shape'] = df['time_shape'].apply(lambda x: int(x.strip('[]')))
    return df

In [38]:
#convert to size(128,1500)
def normalize_shape():
    df = reform_file_data()
    for sample in range(len(df)):
        df.time_shape[sample] = 1500
        df.dims[sample] = list(df.dims[sample][1],1500)
        tmp_sample = torch.load(df.path[sample])
        if list(tmp_sample.shape) == [128,1500]:
            continue
        tmp_sample = tmp_sample.resize(128,tmp_sample.shape[2])
        if df.time_shape[sample] >1500:
            tmp_sample = tmp_sample[:,:1500]
        while tmp_sample.shape[1]<1500:
            last_dims = 1500 - tmp_sample.shape[1]
            if last_dims > tmp_sample.shape[1]:
                tmp_sample = torch.cat((tmp_sample,tmp_sample),dim=1)
            else:
                tmp_sample = torch.cat((tmp_sample,tmp_sample[:,:last_dims]),dim=1)
        torch.save(tmp_sample,df.path[sample])
    df.to_csv('processed_shape_data.csv')

In [39]:
def check_shape():
    df = reform_file_data()
    for sample in range(len(df)):
        tmp_sample = torch.load(df.path[sample])
        if list(tmp_sample.shape) != [128,1500]:
            print(list(tmp_sample.shape))
            return False
    return True

In [40]:
normalize_shape()
check_shape()

C:\Users\hoang\AppData\Local\Temp\ipykernel_16696\1886185345.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time_shape[sample] = 1500


TypeError: list expected at most 1 argument, got 2